### Importing the libraries

- numpy for numerical computations;
- pandas for data analysis;
- matplotlib for plotting;

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Features and dependent variables

In any dataset where you want to train a machine learning model, you have the same entities: features and the dependent variables

- Features - Columns where you want to predict the dependent variable;
- Dependent variable - Column where you will use to train the model to predict them.

In [3]:
dataset = pd.read_csv('Data.csv')

features = dataset.iloc[:, :-1].values
dependent = dataset.iloc[:, -1].values

print(features)
print(dependent)


[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]
['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


### Taking care of the missing data

In [4]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

imputer.fit(features[:, 1:])
features[:, 1:] = imputer.transform(features[:, 1:])

features

array([['France', 44.0, 72000.0],
       ['Spain', 27.0, 48000.0],
       ['Germany', 30.0, 54000.0],
       ['Spain', 38.0, 61000.0],
       ['Germany', 40.0, 63777.77777777778],
       ['France', 35.0, 58000.0],
       ['Spain', 38.77777777777778, 52000.0],
       ['France', 48.0, 79000.0],
       ['Germany', 50.0, 83000.0],
       ['France', 37.0, 67000.0]], dtype=object)

### Encoding categorical data

Since a column in the dataset can contain categorical data, we need to encode it.

One approach would be encode them like numbers, such as:

- France - 1;
- Spain - 2;
- Germany - 3;

But that wouldn't work, since the machine learning algorithm could interpret them as order. So to avoid this, we can use a technique called one-hot encoding.

In [5]:
# Encoding the independent variable
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

column_transformer = ColumnTransformer(
  # Keep the columns that won't be encoded.
  remainder='passthrough',
  transformers=[
    # type of transformation; encoder; column(s) to transform.
    ('encoder', OneHotEncoder(), [0]),
  ],
)

features = np.array(column_transformer.fit_transform(features))

features

array([[1.0, 0.0, 0.0, 44.0, 72000.0],
       [0.0, 0.0, 1.0, 27.0, 48000.0],
       [0.0, 1.0, 0.0, 30.0, 54000.0],
       [0.0, 0.0, 1.0, 38.0, 61000.0],
       [0.0, 1.0, 0.0, 40.0, 63777.77777777778],
       [1.0, 0.0, 0.0, 35.0, 58000.0],
       [0.0, 0.0, 1.0, 38.77777777777778, 52000.0],
       [1.0, 0.0, 0.0, 48.0, 79000.0],
       [0.0, 1.0, 0.0, 50.0, 83000.0],
       [1.0, 0.0, 0.0, 37.0, 67000.0]], dtype=object)

In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

dependent = label_encoder.fit_transform(dependent)

dependent

array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1])

### Splitting the dataset into the training set and testing set

In [7]:
from sklearn.model_selection import train_test_split

features_train, features_test, dependent_train, dependent_test = train_test_split(features, dependent, test_size=0.2, random_state=1)

In [8]:
print(features_train)
print(features_test)
print(dependent_train)
print(dependent_test)

[[0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 35.0 58000.0]]
[[0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 0.0 37.0 67000.0]]
[0 1 0 0 1 1 0 1]
[0 1]


### When is feature scaling applied to the data?

After the splitting of the dataset, to avoid information leakage.

### Feature scaling

The goal of this is to have all the values in the dataset in the same range.

Standardization

- Works well in general

$$x_{stand} = \frac{x - mean(x)}{standard\_deviation(x)}$$

Normalization

- Recommended when you have a normal distribution in most of your features

$$x_{norm} = \frac{x - min(x)}{max(x) - min(x)}$$

It is recommended to go with standardization as a default, since it works for most cases.


In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# [:, 3:] Means that the scaler doesn't take the dummy values (one-hot encoded)
# into the transformation
features_train[:, 3:] = scaler.fit_transform(features_train[:, 3:])

# Only apply the transform method, with the same scaler.
features_test[:, 3:] = scaler.transform(features_test[:, 3:])


In [13]:
features_train

array([[0.0, 0.0, 1.0, -0.19159184384578545, -1.0781259408412425],
       [0.0, 1.0, 0.0, -0.014117293757057777, -0.07013167641635372],
       [1.0, 0.0, 0.0, 0.566708506533324, 0.633562432710455],
       [0.0, 0.0, 1.0, -0.30453019390224867, -0.30786617274297867],
       [0.0, 0.0, 1.0, -1.9018011447007988, -1.420463615551582],
       [1.0, 0.0, 0.0, 1.1475343068237058, 1.232653363453549],
       [0.0, 1.0, 0.0, 1.4379472069688968, 1.5749910381638885],
       [1.0, 0.0, 0.0, -0.7401495441200351, -0.5646194287757332]],
      dtype=object)

In [12]:
features_test

array([[0.0, 1.0, 0.0, -1.4661817944830124, -0.9069571034860727],
       [1.0, 0.0, 0.0, -0.44973664397484414, 0.2056403393225306]],
      dtype=object)